<a href="https://colab.research.google.com/github/DilawarShafiq/assignments/blob/main/OpenRouter_AI_Agents_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [9]:
!pip install -Uq openai-agents

## Provider Config

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [24]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [33]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "thudm/glm-z1-32b:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [38]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "be HIPAA compliant and verify eligibility with Mediare"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1745143628-VwzopmmF9qUfwYkIrOMr', 'provider': 'Chutes', 'model': 'thudm/glm-z1-32b', 'object': 'chat.completion', 'created': 1745143628, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'To achieve **HIPAA compliance** while verifying **Medicare eligibility**, follow this structured approach:\n\n---\n\n### **1. HIPAA Compliance Requirements**\nImplement safeguards across three categories:\n- **Administrative**:\n  - Develop clear policies for data handling, access, and breach reporting.\n  - Conduct regular staff training on HIPAA rules (e.g., minimum necessary principle).\n  - Designate a Privacy Officer and ensure Business Associate Agreements (BAAs) with vendors.\n- **Technical**:\n  - Encrypt all patient data (at rest and in transit) using AES-256 or SHA-256.\n  - Deploy firewalls, intrusion detection systems, and multi-factor authentication.\n  - Log all access and changes to

In [39]:
data = response.json()
data['choices'][0]['message']['content']

'To achieve **HIPAA compliance** while verifying **Medicare eligibility**, follow this structured approach:\n\n---\n\n### **1. HIPAA Compliance Requirements**\nImplement safeguards across three categories:\n- **Administrative**:\n  - Develop clear policies for data handling, access, and breach reporting.\n  - Conduct regular staff training on HIPAA rules (e.g., minimum necessary principle).\n  - Designate a Privacy Officer and ensure Business Associate Agreements (BAAs) with vendors.\n- **Technical**:\n  - Encrypt all patient data (at rest and in transit) using AES-256 or SHA-256.\n  - Deploy firewalls, intrusion detection systems, and multi-factor authentication.\n  - Log all access and changes to patient records for audits.\n- **Physical**:\n  - Restrict physical access to servers/data centers.\n  - Require ID badges/biometrics for entry to sensitive areas.\n\n---\n\n### **2. Medicare Eligibility Verification Process**\n- **Step 1: Use Authorized Tools**  \n  Integrate with HIPAA-com

## 2. Using OpenAI Agents SDK

In [41]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Eligibility Agent",
        instructions="Check eligibility for Medicare patient.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Can you check eligibility for a medicare patient scheduled today?",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

To check the eligibility of a Medicare patient scheduled for **today**, follow these steps:

---

### **1. Gather Patient Information**
   - **Full Name**, Date of Birth (DOB), and:
     - **Medicare Beneficiary Identifier (MBI)** (preferred, 11-character ID introduced in 2021, e.g., `123456789Q`).
     - **Social Security Number (SSN)** (may still be required in some cases, but avoid sharing unnecessarily).
   - **Address/Phone Number**: For plan-specific verification if needed.

---

### **2. Use Medicare’s Official Eligibility Tool**
   - **Website**: [Medicare.gov Eligibility Checker](https://www.medicare.gov/my-medicare/find/care/eligibility-checker)  
     *Steps*:  
     - Go to [Medicare Eligibility](https://www.medicare.gov/my-medicare/find/care/eligibility-checker).  
     - Enter the patient’s MBI (if available) or SSN + DOB.  
     - Review results for **active enrollment** in Parts A/B (if the service requires them) and any plan-specific rules (e.g., Medicare Advantage).  

# OpenRouter 404 Error Solution

## Error - No endpoints found matching
```python
NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy. Enable prompt training here: https://openrouter.ai/settings/privacy', 'code': 404}}
```

## Cause
This error occurs when OpenRouter API can't find endpoints matching your data policy, typically because prompt training is disabled.

## Solution

1. **Enable Prompt Training**:
   - Visit [OpenRouter Privacy Settings](https://openrouter.ai/settings/privacy)
   - Toggle ON "Prompt Training" option

2. **Re-run your code** after enabling

![OpenRouter Settings Screenshot](./openrouter.png)
*(Example: Enable prompt training in privacy settings)*

## Prevention
Keep prompt training enabled for uninterrupted API access.
```